In [1]:
!pip install pandas sqlalchemy sqlacodegen

## Storing Data with Pickles

In [2]:
import pickle
import pandas as pd

In [3]:
data = [{"name": "Justin"}]

with open("users.pkl", 'wb') as f:
    pickle.dump(data, f) # pickle.dump

In [4]:
data_from_pkl = pickle.load(open('users.pkl', 'rb')) # pickle.loads -> pickle.load
print(data_from_pkl, type(data_from_pkl))

[{'name': 'Justin'}] <class 'list'>


In [5]:
df = pd.DataFrame(data)
df.head()

,name
0,Justin


In [6]:
df.to_pickle("df_pkl")
df.dtypes

name    object
dtype: object

In [7]:
df2 = pd.read_pickle("df_pkl")
df2.head()

,name
0,Justin


In [8]:
df2.dtypes

name    object
dtype: object

In [9]:
df.to_csv("my_df.csv", index=False)

## SQLAlchemy

SQLAlchmey & Django have a similar object-relational mappings (aka `ORM`) are ways to connect Python to a SQL database.

In [10]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str = 'Unknown'
    genre:str = 'Action'
    year:int = None
    
# class Movie:
#     name = 'Unknown'
#     genre = 'Action'
    
#     def __init__(self, name='', *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.name = name

In [11]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
movie_obj.name

'Interstellar'

In [12]:
data = [{
    "name": "Interstellar",
    "genre": "Sci-Fi"
},
{
    "name": "The Martian",
    "genre": "Sci-Fi"
},
{
    "name": "Arrival",
    "genre": "Sci-Fi"
}
]

In [13]:
df = pd.DataFrame(data)
df.head()

,name,genre
0,Interstellar,Sci-Fi
1,The Martian,Sci-Fi
2,Arrival,Sci-Fi


In [14]:
import sqlalchemy
from sqlalchemy import Column, Integer, String
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import sessionmaker

In [15]:
engine = create_engine("sqlite:///app.db") # mysql, postgres

In [16]:
Session = sessionmaker(bind=engine)
my_sess = Session()

In [17]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [18]:
class Movie(Base): # table
    __tablename__ = "movies"
    
    id = Column(Integer, primary_key=True) # auto created for us
    name = Column(String) # 'Unknown' # col
    genre = Column(String) # 'Action' # col
    description = Column(String) # 'Action' # col
    year = Column(Integer, nullable=True) # None # col
    
    
    def __repr__(self):
        return f"<Movie name={self.name}>"
        
# saved instance = row

In [19]:
# Add table(s) to database
Base.metadata.create_all(engine)

In [20]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
print(movie_obj.name)

Interstellar


In [21]:
movie_obj.id

In [22]:
my_sess.add(movie_obj) # prepare to save
my_sess.commit() # save

In [23]:
print(movie_obj.id, movie_obj.description)

7 None


In [24]:
movie_obj2 = Movie(name='The Martian', genre='Sci-Fi')
print(movie_obj2.name, movie_obj2.id)

The Martian None


In [25]:
my_sess.add(movie_obj2)

In [26]:
movie_obj3 = Movie(name='Inception', genre='Sci-Fi')
print(movie_obj3.name, movie_obj3.id)

Inception None


In [27]:
my_sess.add(movie_obj3)

In [28]:
my_sess.commit()

In [29]:
print(movie_obj2.id, movie_obj3.id)

8 9


## CRUD in SQLAlchemy

- **C**reate
- **R**etreive
- **U**pdate
- **D**elete

In [30]:
Session = sessionmaker(bind=engine)
session = Session()

In [31]:
# Create
movie = Movie(name='Guaridans of the Galaxy')
session.add(movie)
session.commit()

In [32]:
# Retreive

# Get 1 item
movie_a = session.query(Movie).get(5)
print(movie_a.id, movie_a.name, movie_a.description)

5 Inception A dream within a dream


In [33]:
# List
qs = session.query(Movie).all()
print(qs)

[<Movie name=Guardians of the Galaxy>, <Movie name=Interstellar>, <Movie name=The Martian>, <Movie name=Inception>, <Movie name=Guardians of the Galaxy>, <Movie name=Interstellar>, <Movie name=The Martian>, <Movie name=Inception>, <Movie name=Guaridans of the Galaxy>]


In [34]:
# List & Filter by Column Value
qs = session.query(Movie).filter_by(name='Guardians of the Galaxy').all()
qs

[<Movie name=Guardians of the Galaxy>, <Movie name=Guardians of the Galaxy>]

In [35]:
# List & Filter by Column Value Containing Something
qs = session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
qs

[<Movie name=Guaridans of the Galaxy>]

In [36]:
# List & Filter by Column Value Containing Something
my_query = input("What are you looking for?\n") or "Unknown"
qs = session.query(Movie).filter(Movie.name.contains(my_query)).all()
print(qs)

What are you looking for?
Guardians
[<Movie name=Guardians of the Galaxy>, <Movie name=Guardians of the Galaxy>]


In [37]:
# Update

movie_a = session.query(Movie).get(5)
movie_a.description = "A dream within a dream"
print(movie_a.id, movie_a.name, movie_a.description)
session.commit()

5 Inception A dream within a dream


In [38]:
movie_a = session.query(Movie).get(5)
print(movie_a.description)

A dream within a dream


In [39]:
qs = session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
for movie_obj in qs:
    movie_obj.name = "Guardians of the Galaxy"
session.commit()

In [40]:
qs = session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
qs

[]

In [41]:
# Delete
movie_a = session.query(Movie).get(1)
session.delete(movie_a)
session.commit()

UnmappedInstanceError: Class 'builtins.NoneType' is not mapped

In [42]:
session.flush()

## SQLAlchemy & Pandas

In [43]:
!sqlacodegen sqlite:///app2.db

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)


In [44]:
engine = create_engine("sqlite:///app2.db")
Session = sessionmaker(bind=engine)
session = Session()

In [45]:
# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class MovieApp2(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return f"<Movie name={self.name}>"

In [46]:
qs = session.query(MovieApp2).all()
print(qs)

[<Movie name=Interstellar>, <Movie name=The Martian>, <Movie name=Inception>]


In [51]:
for old_obj in qs:
    movie_obj = Movie(name=old_obj.name, genre=old_obj.genre, year=old_obj.year)
    print(movie_obj.name)
    my_sess.add(movie_obj)
my_sess.commit()

Interstellar
The Martian
Inception


In [52]:
old_engine = create_engine("sqlite:///app2.db")
old_df = pd.read_sql_table("movies", old_engine)
old_df.head()

,id,name,genre,year
0,1,Interstellar,Sci-Fi,None
1,2,The Martian,Sci-Fi,None
2,3,Inception,Sci-Fi,None


In [53]:
current_engine = create_engine("sqlite:///app.db")
current_df = pd.read_sql_table("movies", current_engine)
current_df.head()

,id,name,genre,description,year
0,2,Guardians of the Galaxy,None,None,None
1,3,Interstellar,Sci-Fi,None,None
2,4,The Martian,Sci-Fi,None,None
3,5,Inception,Sci-Fi,A dream within a dream,None
4,6,Guardians of the Galaxy,None,None,None


In [65]:
final_df = pd.concat([current_df, old_df])
final_df.head(n=20)

,id,name,genre,description,year
0,2,Guardians of the Galaxy,None,None,None
1,3,Interstellar,Sci-Fi,None,None
2,4,The Martian,Sci-Fi,None,None
3,5,Inception,Sci-Fi,A dream within a dream,None
4,6,Guardians of the Galaxy,None,None,None
5,7,Interstellar,Sci-Fi,None,None
6,8,The Martian,Sci-Fi,None,None
7,9,Inception,Sci-Fi,None,None
8,10,Guardians of the Galaxy,None,None,None
9,11,Interstellar,Sci-Fi,None,None


In [67]:
final_df = final_df[['name', 'genre', 'description', 'year']]
final_df.reset_index(inplace=True, drop=True)
final_df.head(n=20)

,name,genre,description,year
0,Guardians of the Galaxy,None,None,None
1,Interstellar,Sci-Fi,None,None
2,The Martian,Sci-Fi,None,None
3,Inception,Sci-Fi,A dream within a dream,None
4,Guardians of the Galaxy,None,None,None
5,Interstellar,Sci-Fi,None,None
6,The Martian,Sci-Fi,None,None
7,Inception,Sci-Fi,None,None
8,Guardians of the Galaxy,None,None,None
9,Interstellar,Sci-Fi,None,None


In [72]:
final_df.to_sql(
    "movies",
    current_engine,
    if_exists='replace',
    dtype = {
        "name": String,
        "genre": String,
        "year": Integer,
        "description": String,
    }
)

In [73]:
!sqlacodegen sqlite:///app.db

# coding: utf-8
from sqlalchemy import BigInteger, Column, Integer, MetaData, String, Table

metadata = MetaData()


t_movies = Table(
    'movies', metadata,
    Column('index', BigInteger, index=True),
    Column('name', String),
    Column('genre', String),
    Column('description', String),
    Column('year', Integer)
)


t_movies_2 = Table(
    'movies_2', metadata,
    Column('index', BigInteger, index=True),
    Column('name', String),
    Column('genre', String),
    Column('description', String),
    Column('year', Integer)
)


In [76]:
import sqlite3


conx = sqlite3.connect('app.db')
conx2 = create_engine("sqlite:///app.db")

df = pd.read_sql_query("SELECT * FROM movies", conx2)
df.head()

,index,name,genre,description,year
0,0,Guardians of the Galaxy,None,None,None
1,1,Interstellar,Sci-Fi,None,None
2,2,The Martian,Sci-Fi,None,None
3,3,Inception,Sci-Fi,A dream within a dream,None
4,4,Guardians of the Galaxy,None,None,None


In [ ]:
# cfe.sh/youtube